<a href="https://colab.research.google.com/github/reisneriv/NBA-Win-Predictor/blob/main/NBA_Free_Agency_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import BayesianRidge, ElasticNet, LinearRegression, SGDRegressor, LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm, preprocessing, datasets, linear_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
# Used for data storage and manipulation
import numpy as np
from sklearn import linear_model, datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import sklearn.metrics as sm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from seaborn import pairplot

In [ ]:
import io
import os
import pandas as pd
import numpy as np
import decimal

pd.options.display.max_rows = 999

#Reading in the training file I made
df = pd.read_csv("/content/Train_Free_agents.csv")
train = df
#Specifying the data points of interest that I'm running the models for
train['% of Sal Cap'] = train['% of Sal Cap']*100
x_train = train.drop(["Value","Yrs","Avg. Salary", "% of Sal Cap", "Max Value"], axis = 1)#,"Yrs", "Avg. Salary", "% of Sal Cap", "Max Value"
y_train = train["% of Sal Cap"]
y_train=y_train.astype('int')
z_train = train["Yrs"]
z_train=z_train.astype('int')
w_train = train["Value"]
w_train=w_train.astype('int')
#Reading in the testing data which is 2023 free agents
test = pd.read_csv("/content/Test_Free_agents.csv")
test['% of Sal Cap'] = test['% of Sal Cap']*100
x_test = test.drop(["Value","Player","Yrs","Avg. Salary", "% of Sal Cap", "Max Value"], axis =1) #"Yrs","Avg. Salary", "% of Sal Cap", "Max Value",
y_test = test["% of Sal Cap"]
y_test=y_test.astype('int')
z_test = test["Yrs"]
z_test=z_test.astype('int')
w_test = test["Value"]
w_test=w_test.astype('int')
train.head(180)
#fitting the data sets to work within all of the models
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
corr = train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 53 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pos.          882 non-null    int64  
 1   Age           882 non-null    float64
 2   Type          882 non-null    int64  
 3   Yrs           882 non-null    int64  
 4   Value         882 non-null    int64  
 5   Avg. Salary   882 non-null    float64
 6   % of Sal Cap  882 non-null    float64
 7   Max Value     882 non-null    int64  
 8   G             882 non-null    int64  
 9   GS            882 non-null    int64  
 10  MP            882 non-null    float64
 11  FG            882 non-null    float64
 12  FGA           882 non-null    float64
 13  FG%           882 non-null    float64
 14  3P            882 non-null    float64
 15  3PA           882 non-null    float64
 16  3P%           882 non-null    float64
 17  2P            882 non-null    float64
 18  2PA           882 non-null    

In [ ]:
#Finding significant correlation information for the salaries offered
corr = train.corr()
Wcorr = corr["% of Sal Cap"]
corr = train.corr()
Ycorr = corr["Yrs"]
Zcorr = corr["Value"]
Xcorr = corr["Avg. Salary"]

print(Wcorr)
print(Ycorr)
print(Zcorr)
print(Xcorr)
ix = train.corr().sort_values("Value", ascending=False).index
print(ix)

Pos.            0.030032
Age             0.018970
Type           -0.087624
Yrs             0.740775
Value           0.937012
Avg. Salary     0.988399
% of Sal Cap    1.000000
Max Value       0.936952
G               0.404219
GS              0.642373
MP              0.639021
FG              0.751138
FGA             0.723248
FG%             0.147165
3P              0.500171
3PA             0.473601
3P%             0.165864
2P              0.658279
2PA             0.666338
2P%             0.109883
eFG%            0.168663
FT              0.671190
FTA             0.632244
FT%             0.195554
ORB             0.239710
DRB             0.531321
TRB             0.471131
AST             0.526359
STL             0.495466
BLK             0.283104
TOV             0.581220
PF              0.358957
PTS             0.761996
PER             0.221148
TS%             0.118650
3PAr           -0.027357
FTr             0.010497
ORB%           -0.007808
DRB%            0.083473
TRB%            0.055683


In [ ]:
#sns.pairplot(df, x_vars=['FG','3P', '2P','TRB','TOV','PTS','DefFG', 'Def3P', 'DefTRB', 'DefTOV', 'DefPTS'], y_vars='W', height=7,aspect=0.7)
#sns.pairplot(train, x_vars=['NRtg','ORtg', 'FG%', '3P%'], y_vars = 'W', height = 7, aspect = 0.7)
#sns.regplot(x='NRtg', y= 'W', data = df)


In [ ]:
#sns.pairplot(train, x_vars=['2P%', 'Def2P%', 'Def3P%', 'DRtg', 'DefFG%'], y_vars = 'W', height = 7, aspect = 0.7)


In [ ]:
#g=sns.pairplot(train, x_vars=['NRtg', 'MOV', 'SRS', 'ORtg', 'TS%', 'FG%', 'OffeFG%', '3P%',
#       '2P%', 'DRB', 'PTS', 'FG', 'TRB', 'BLK', 'AST', '3P', 'DefDRB%', 'FT%',
#       '3PAr', 'OffFT/FGA', 'FT', '3PA', 'STL', 'OffORB%', 'DefPF', 'FTr',
#       'FTA', '2P', 'Def2PA', 'DefFGA', 'DefTOV%', 'DefTOV', 'Def3PA', 'FGA',
#       'Pace', 'DefORB', 'ORB', 'DefFT%', 'PF', 'Def3P', 'DefFTA', '2PA',
#       'DefFT/FGA', 'DefFT', 'OffTOV%', 'TOV', 'Def2P', 'DefSTL', 'DefAST',
#       'DefFG', 'DefPTS', 'DefTRB', 'DefDRB', 'SOS', 'Def2P%', 'DefBLK',
#       'Def3P%', 'DefeFG%', 'DRtg', 'DefFG%', 'L', 'G'], y_vars = 'W', height = 7, aspect = 0.7, diag_kind ="kde")
#g.map_lower(sns.kdeplot, levels = 4, color = ".2")

In [ ]:
X, y =train[['Pos.', 'Age',	'Type',	'Yrs',	'Value',
             '% of Sal Cap',	'Max Value',	'G',	'GS',	'MP',	'FG',	'FGA',
             'FG%',	'3P',	'3PA',	'3P%',	'2P',	'2PA',	'2P%',	'eFG%',	'FT',
             'FTA',	'FT%',	'ORB',	'DRB',	'TRB',	'AST',	'STL',	'BLK',
             'TOV',	'PF',	'PTS',	'PER',	'TS%',	'3PAr',	'FTr',	'ORB%',
             'DRB%',	'TRB%',	'AST%',	'STL%',	'BLK%',	'TOV%',	'USG%',	'OWS',
             'DWS',	'WS',	'WS/48',	'OBPM',	'DBPM',	'BPM',	'VORP']], train["Avg. Salary"]

br = BayesianRidge()
en = ElasticNet()
gbr = GradientBoostingRegressor()
lgbm = LGBMRegressor()
lin = LinearRegression()
mlp = MLPRegressor(max_iter=1000)
sgd = SGDRegressor()
#svr = SVR(kernel="linear")

br_scores = cross_val_score(br, X, y, cv=10)
en_scores = cross_val_score(en, X, y, cv=10)
gbr_scores = cross_val_score(gbr, X, y, cv=10)
lgbm_scores = cross_val_score(lgbm, X, y, cv=10)
lin_scores = cross_val_score(lin, X, y, cv=10)
mlp_scores = cross_val_score(mlp, X, y, cv=10)
sgd_scores = cross_val_score(sgd, X, y, cv=10)
#svr_scores = cross_val_score(svr, X, y, cv=10)

print("BaynesianRidge:           ", br_scores.mean(), br_scores.std())
print("ElasticNet:               ", en_scores.mean(), en_scores.std())
print("GradientBoostingRegressor:", gbr_scores.mean(), gbr_scores.std())
print("LGBMRegressor:            ", lgbm_scores.mean(), lgbm_scores.std())
print("LinearRegression:         ", lin_scores.mean(), lin_scores.std())
print("MLPRegressor:             ", mlp_scores.mean(), mlp_scores.std())
print("SGDRegressor:             ", sgd_scores.mean(), sgd_scores.std())
#print("SVR:                      ", svr_scores.mean(), svr_scores.std())

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.874e+14, tolerance: 4.347e+12
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.210e+14, tolerance: 4.023e+12
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.987e+14, tolerance: 4.189e

BaynesianRidge:            0.8903262486464676 0.040902449766603906
ElasticNet:                0.9754662234552847 0.013759010484191599
GradientBoostingRegressor: 0.9757695276962217 0.015761094939936934
LGBMRegressor:             0.9736021613801071 0.00967283577706324
LinearRegression:          0.9755723664846732 0.014164553120970735
MLPRegressor:              0.8444103969591911 0.0594003712896284
SGDRegressor:              -6.134462220464374e+34 1.378679009471968e+35


In [ ]:
scaler = StandardScaler()

# Standardize the dataset.
train_std = pd.DataFrame(scaler.fit_transform(train))
train_std.columns = train.columns

# Prepare the dataset for supervised learning.
X_std, y_std =train_std[['Pos.', 'Age',	'Type',	'Yrs',	'Avg. Salary',
             '% of Sal Cap',	'Max Value',	'G',	'GS',	'MP',	'FG',	'FGA',
             'FG%',	'3P',	'3PA',	'3P%',	'2P',	'2PA',	'2P%',	'eFG%',	'FT',
             'FTA',	'FT%',	'ORB',	'DRB',	'TRB',	'AST',	'STL',	'BLK',
             'TOV',	'PF',	'PTS',	'PER',	'TS%',	'3PAr',	'FTr',	'ORB%',
             'DRB%',	'TRB%',	'AST%',	'STL%',	'BLK%',	'TOV%',	'USG%',	'OWS',
             'DWS',	'WS',	'WS/48',	'OBPM',	'DBPM',	'BPM',	'VORP']], train["Value"]

# Evaluate the models with cross validation.
br_std_scores = cross_val_score(br, X_std, y_std, cv=10)
en_std_scores = cross_val_score(en, X_std, y_std, cv=10)
gbr_std_scores = cross_val_score(gbr, X_std, y_std, cv=10)
lgbm_std_scores = cross_val_score(lgbm, X_std, y_std, cv=10)
lin_std_scores = cross_val_score(lin, X_std, y_std, cv=10)
mlp_std_scores = cross_val_score(mlp, X_std, y_std, cv=10)
sgd_std_scores = cross_val_score(sgd, X_std, y_std, cv=10)
#svr_std_scores = cross_val_score(svr, X_std, y_std, cv=10)

print("BaynesianRidge:           ", br_scores.mean(), br_scores.std())
print("ElasticNet:               ", en_scores.mean(), en_scores.std())
print("GradientBoostingRegressor:", gbr_scores.mean(), gbr_scores.std())
print("LGBMRegressor:            ", lgbm_scores.mean(), lgbm_scores.std())
print("LinearRegression:         ", lin_scores.mean(), lin_scores.std())
print("MLPRegressor:             ", mlp_scores.mean(), mlp_scores.std())
print("SGDRegressor:             ", sgd_scores.mean(), sgd_scores.std())
#print("SVR:                      ", svr_scores.mean(), svr_scores.std())

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perce

BaynesianRidge:            0.8903262486464676 0.040902449766603906
ElasticNet:                0.9754662234552847 0.013759010484191599
GradientBoostingRegressor: 0.9757695276962217 0.015761094939936934
LGBMRegressor:             0.9736021613801071 0.00967283577706324
LinearRegression:          0.9755723664846732 0.014164553120970735
MLPRegressor:              0.8444103969591911 0.0594003712896284
SGDRegressor:              -6.134462220464374e+34 1.378679009471968e+35


In [ ]:
print(train.shape)
#Nancount = x_test.isna().sum().sum()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(882, 53)
(882, 48)
(882,)
(197, 48)
(197,)


In [ ]:
linereg = linear_model.LinearRegression()
linereg.fit(x_train, y_train)
Y_predict_linear_regression = linereg.predict(x_test)
#Y_predict_linear_regression
linereg.fit(x_train, z_train)
Z_predict_linear_regression = linereg.predict(x_test)
linereg.fit(x_train, w_train)
W_predict_linear_regression = linereg.predict(x_test)



order =[]
for i in range(197):
  #print(test['Player'][i], decimal.Decimal(Y_predict_linear_regression[i]))
  #order.append([test['Player'][i], decimal.Decimal(Y_predict_linear_regression[i])])
  order.append([test['Player'][i], Y_predict_linear_regression[i], Z_predict_linear_regression[i], W_predict_linear_regression[i]])


df = pd.DataFrame(order)

# Save the DataFrame to an Excel file
df.to_excel('linReg_predict_Combined.xlsx', index=False)
#df.to_csv

In [ ]:
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, Y_predict_linear_regression), 2))
print("R2 score =", round(sm.r2_score(y_test, Y_predict_linear_regression), 2))


Mean absolute error = 4.74
R2 score = 0.0


In [ ]:
DT_model = DecisionTreeRegressor(max_depth=None).fit(x_train,y_train)
DT_predict = DT_model.predict(x_test) #Predictions on Testing data
#DT_predict = DT_predict.round() for years
DT_predict = DT_predict
#print(DT_predict)

In [ ]:
order =[]
for i in range(197):
  #print(test['Player'][i], DT_predict[i])
  order.append([test['Player'][i], DT_predict[i]])

df = pd.DataFrame(order)

# Save the DataFrame to an Excel file
df.to_excel('decision_tree_predict_%ofCap.xlsx', index=False)
#df.to_csv

In [ ]:
KNN_model = KNeighborsRegressor(n_neighbors=5).fit(x_train,y_train)
KNN_predict = KNN_model.predict(x_test) #Predictions on Testing data
#KNN_predict = KNN_predict.round() for years
KNN_predict = KNN_predict
#print(KNN_predict)

In [ ]:
order =[]
for i in range(197):
  #print(test['Player'][i], KNN_predict[i])
  order.append([test['Player'][i], KNN_predict[i]])
df = pd.DataFrame(order)
df.to_excel('KNN_predict_%ofCap.xlsx', index=False)
